In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Intellectual_Disability"
cohort = "GSE59630"

# Input paths
in_trait_dir = "../../input/GEO/Intellectual_Disability"
in_cohort_dir = "../../input/GEO/Intellectual_Disability/GSE59630"

# Output paths
out_data_file = "../../output/preprocess/Intellectual_Disability/GSE59630.csv"
out_gene_data_file = "../../output/preprocess/Intellectual_Disability/gene_data/GSE59630.csv"
out_clinical_data_file = "../../output/preprocess/Intellectual_Disability/clinical_data/GSE59630.csv"
json_path = "../../output/preprocess/Intellectual_Disability/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
# 1. Check gene expression data availability
# This dataset is about gene expression in the brain, as mentioned in the background information
is_gene_available = True

# 2. Variable availability and data type conversion
# 2.1 Data Availability

# Intellectual Disability is represented as Down Syndrome (DS) in this dataset
# Found in key 2: 'disease status: CTL', 'disease status: DS'
trait_row = 2  

# Age information is in key 4
age_row = 4  

# Gender information is in key 3: 'Sex: F', 'Sex: M'
gender_row = 3  

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value (Down Syndrome status) to binary (0 for control, 1 for DS)"""
    if value is None:
        return None
    # Extract the value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'DS':
        return 1
    elif value.upper() == 'CTL':
        return 0
    return None

def convert_age(value):
    """Convert age to a numeric value in years"""
    if value is None:
        return None
    
    # Extract the value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric part and unit
    import re
    match = re.match(r'(\d+)(\w+)', value)
    if not match:
        return None
    
    number, unit = match.groups()
    number = float(number)
    
    # Convert to years based on unit
    if unit == 'yr':
        return number
    elif unit == 'mo':
        return number / 12
    elif unit == 'wg':  # weeks of gestation
        return number / 52  # approximate conversion to years
    
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract the value after colon and strip whitespace
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    
    return None

# 3. Save Metadata
# Since trait_row is not None, trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    sample_chars = {
        0: ['braincode: 97-DFC', 'braincode: 100-DFC', 'braincode: 100-V1C', 'braincode: 159-V1C', 'braincode: 132-DFC', 'braincode: 132-V1C', 'braincode: 132-CBC', 'braincode: 131-OFC', 'braincode: 131-DFC', 'braincode: 131-VFC', 'braincode: 131-ITC', 'braincode: 131-V1C', 'braincode: 131-HIP', 'braincode: 131-CBC', 'braincode: 171-DFC', 'braincode: 171-VFC', 'braincode: 171-MFC', 'braincode: 171-OFC', 'braincode: 171-S1C', 'braincode: 171-IPC', 'braincode: 171-STC', 'braincode: 171-ITC', 'braincode: 171-V1C', 'braincode: 171-CBC', 'braincode: 122-V1C', 'braincode: 122-CBC', 'braincode: 143-OFC', 'braincode: 143-DFC', 'braincode: 173-VFC', 'braincode: 173-ITC'], 
        1: ['region: DFC', 'region: V1C', 'region: CBC', 'region: OFC', 'region: VFC', 'region: ITC', 'region: HIP', 'region: MFC', 'region: S1C', 'region: IPC', 'region: STC', 'region: FC'], 
        2: ['disease status: CTL', 'disease status: DS'], 
        3: ['Sex: F', 'Sex: M'], 
        4: ['age: 17wg', 'age: 19wg', 'age: 22wg', 'age: 4mo', 'age: 6mo', 'age: 10mo', 'age: 12mo', 'age: 2yr', 'age: 3yr', 'age: 8yr', 'age: 15yr', 'age: 18yr', 'age: 22yr', 'age: 30yr', 'age: 42yr', 'age: 16wg', 'age: 1mo', 'age: 9mo', 'age: 14mo', 'age: 10yr', 'age: 13yr', 'age: 19yr', 'age: 39yr', 'age: 40yr'], 
        5: ['Stage: 5', 'Stage: 6', 'Stage: 8', 'Stage: 9', 'Stage: 10', 'Stage: 11', 'Stage: 12', 'Stage: 13', 'Stage: 14'], 
        6: ['postmortem interval: 2', 'postmortem interval: 4', 'postmortem interval: 1', 'postmortem interval: 22', 'postmortem interval: 26', 'postmortem interval: 18', 'postmortem interval: 12', 'postmortem interval: 8', 'postmortem interval: 16', 'postmortem interval: 14.5', 'postmortem interval: 28', 'postmortem interval: 9.5', 'postmortem interval: 19', 'postmortem interval: 3', 'postmortem interval: 23', 'postmortem interval: 24', 'postmortem interval: 11', 'postmortem interval: 17', 'postmortem interval: 15', 'postmortem interval: 10'], 
        7: ['race: CC', 'race: AA', 'race: H'], 
        8: ['rna integrity number: 8.9', 'rna integrity number: 10', 'rna integrity number: 7.6', 'rna integrity number: 9.5', 'rna integrity number: 9.2', 'rna integrity number: 7', 'rna integrity number: 7.1', 'rna integrity number: 7.5', 'rna integrity number: 7.4', 'rna integrity number: 8.4', 'rna integrity number: 8.6', 'rna integrity number: 8.8', 'rna integrity number: 8.7', 'rna integrity number: 9.4', 'rna integrity number: 9', 'rna integrity number: 9.6', 'rna integrity number: 9.8', 'rna integrity number: 5', 'rna integrity number: 7.2', 'rna integrity number: 8', 'rna integrity number: 7.8', 'rna integrity number: 9.7', 'rna integrity number: 8.1', 'rna integrity number: 7.9', 'rna integrity number: 9.3', '


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
import pandas as pd
import json
import os
import re
from typing import Optional, Callable, Dict, Any, List

# First, examine what files are available in the cohort directory
print(f"Examining directory: {in_cohort_dir}")
if os.path.exists(in_cohort_dir):
    directory_contents = os.listdir(in_cohort_dir)
    print(f"Files in directory: {directory_contents}")
else:
    print(f"Directory {in_cohort_dir} does not exist")
    directory_contents = []

# Check for available data files
series_matrix_files = [f for f in directory_contents if "series_matrix" in f.lower()]
soft_files = [f for f in directory_contents if f.lower().endswith(".soft") or f.lower().endswith(".soft.gz")]
json_files = [f for f in directory_contents if f.lower().endswith(".json")]

# Load clinical data from available files
clinical_data_dict = {}
sample_characteristics = {}

# Try to load clinical data from different possible file formats
if json_files and any("clinical_data" in f.lower() for f in json_files):
    clinical_json = next(f for f in json_files if "clinical_data" in f.lower())
    clinical_data_path = os.path.join(in_cohort_dir, clinical_json)
    try:
        with open(clinical_data_path, 'r') as f:
            clinical_data_dict = json.load(f)
        sample_characteristics = clinical_data_dict.get("sample_characteristics", {})
        print(f"Loaded clinical data from {clinical_json}")
    except Exception as e:
        print(f"Error loading {clinical_json}: {str(e)}")
elif series_matrix_files:
    # We would need to parse series matrix file to extract clinical data
    print("Series matrix files found but need additional parsing")
    # Placeholder for series matrix parsing
    is_gene_available = True
    is_trait_available = False
elif soft_files:
    # We would need to parse soft file to extract clinical data
    print("SOFT files found but need additional parsing")
    # Placeholder for SOFT file parsing
    is_gene_available = True
    is_trait_available = False
else:
    print("No recognizable clinical data files found")
    is_gene_available = False
    is_trait_available = False

# If we have sample characteristics, analyze them
if sample_characteristics:
    print("Sample characteristics keys:")
    for key in sample_characteristics.keys():
        print(f"Key {key}: {sample_characteristics[key]}")

    # Determine if gene expression data is available
    platform_id = clinical_data_dict.get("platform_id", "")
    title = clinical_data_dict.get("title", "")
    summary = clinical_data_dict.get("summary", "")

    is_gene_available = True
    if any(term in platform_id.lower() or term in title.lower() or term in summary.lower() 
           for term in ["mirna", "methylation", "methyl"]):
        if not any(term in platform_id.lower() or term in title.lower() or term in summary.lower() 
                  for term in ["gene expression", "transcriptome", "mrna"]):
            is_gene_available = False

    # Identify the row in sample characteristics containing trait information
    trait_row = None
    age_row = None
    gender_row = None

    # Check each key in sample characteristics for trait, age, and gender data
    for key, values in sample_characteristics.items():
        unique_values = set(values)
        
        # Check for intellectual disability information
        if any("intellectual" in str(v).lower() for v in unique_values) or \
           any("disability" in str(v).lower() for v in unique_values) or \
           any("id" in str(v).lower() and "patient" in str(v).lower() for v in unique_values) or \
           any("control" in str(v).lower() for v in unique_values):
            if len(unique_values) > 1:  # Ensure it's not a constant feature
                trait_row = int(key)
        
        # Check for age information
        if any("age" in str(v).lower() for v in unique_values):
            if len(unique_values) > 1:  # Ensure it's not a constant feature
                age_row = int(key)
        
        # Check for gender information
        if any("gender" in str(v).lower() for v in unique_values) or \
           any("sex" in str(v).lower() for v in unique_values) or \
           any("male" in str(v).lower() for v in unique_values) or \
           any("female" in str(v).lower() for v in unique_values):
            if len(unique_values) > 1:  # Ensure it's not a constant feature
                gender_row = int(key)

    # Define conversion functions
    def convert_trait(value):
        """Convert trait values to binary (0: control, 1: intellectual disability)."""
        if value is None:
            return None
        
        value_lower = str(value).lower()
        
        # Extract content after colon if present
        if ":" in value_lower:
            value_lower = value_lower.split(":", 1)[1].strip()
        
        if any(term in value_lower for term in ["patient", "case", "intellectual disability", "id patient"]):
            return 1
        elif any(term in value_lower for term in ["control", "healthy", "normal"]):
            return 0
        
        return None

    def convert_age(value):
        """Convert age values to continuous numeric values."""
        if value is None:
            return None
        
        value_str = str(value)
        
        # Extract content after colon if present
        if ":" in value_str:
            value_str = value_str.split(":", 1)[1].strip()
        
        # Try to extract numeric age using regex
        age_match = re.search(r'(\d+(?:\.\d+)?)', value_str)
        if age_match:
            return float(age_match.group(1))
        
        return None

    def convert_gender(value):
        """Convert gender values to binary (0: female, 1: male)."""
        if value is None:
            return None
        
        value_lower = str(value).lower()
        
        # Extract content after colon if present
        if ":" in value_lower:
            value_lower = value_lower.split(":", 1)[1].strip()
        
        if any(term in value_lower for term in ["female", "f", "woman"]):
            return 0
        elif any(term in value_lower for term in ["male", "m", "man"]):
            return 1
        
        return None

    # Check trait data availability
    is_trait_available = trait_row is not None
else:
    # If no sample characteristics data is found, set variables to default values
    trait_row = None
    age_row = None
    gender_row = None
    is_trait_available = False

# Print found information
print(f"Gene expression data available: {is_gene_available}")
print(f"Trait data available: {is_trait_available}")
if is_trait_available:
    print(f"Trait row: {trait_row}")
    print(f"Age row: {age_row}")
    print(f"Gender row: {gender_row}")

# Save metadata with initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available and sample_characteristics:
    # Convert sample characteristics dictionary to DataFrame for processing
    clinical_df = pd.DataFrame.from_dict(sample_characteristics, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_df,
        trait=trait,
        trait_row=trait_row,


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


### Step 5: Gene Identifier Review

In [ ]:
# Examining the gene identifiers from the output
# These appear to be Affymetrix probe IDs, not standard human gene symbols
# These numeric IDs (2315554, 2315633, etc.) need to be mapped to gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identifying mapping columns in the gene annotation data
print("Examining gene annotation data to identify mapping columns...")

# The 'ID' column in gene_annotation contains probe IDs matching the gene expression data indices
# The 'gene_assignment' column contains gene symbol information, but needs parsing

# Examining the gene_assignment column
print("\nExample gene_assignment values:")
print(gene_annotation['gene_assignment'].head(3).tolist())

# 2. Create a mapping dataframe with probe IDs and gene symbols
print("\nCreating gene mapping dataframe...")

# Function to extract gene symbols from gene_assignment string
def extract_gene_symbols(assignment):
    if isinstance(assignment, str) and assignment != '---':
        # Extract gene symbols that appear after '//' (typically the second element after splitting by '//')
        parts = assignment.split('//')
        if len(parts) >= 2:
            gene_symbol = parts[1].strip()
            return gene_symbol
    return None

# Apply extraction to create mapping
gene_annotation['Gene'] = gene_annotation['gene_assignment'].apply(extract_gene_symbols)

# Create mapping dataframe with ID and Gene columns
mapping_df = gene_annotation[['ID', 'Gene']].copy()
mapping_df = mapping_df.dropna()  # Remove rows without gene symbols
print(f"Created mapping with {len(mapping_df)} entries")
print("\nMapping preview:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("\nApplying gene mapping to convert probe measurements to gene expression...")

# Use the library function to map probes to genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"Converted data to {len(gene_data.index)} unique genes")
print("\nFirst few gene symbols:")
print(gene_data.index[:10].tolist())

# Print shape of resulting gene expression matrix
print(f"\nGene expression matrix shape: {gene_data.shape}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Get gene mapping function from previous step
def extract_gene_symbols(assignment):
    if isinstance(assignment, str) and assignment != '---':
        # Extract gene symbols that appear after '//' (typically the second element after splitting by '//')
        parts = assignment.split('//')
        if len(parts) >= 2:
            gene_symbol = parts[1].strip()
            return gene_symbol
    return None

# 2. Get the file paths for the SOFT file and matrix file again
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 3. Retrieve and process gene data 
print("Retrieving gene expression data...")
gene_data_raw = get_genetic_data(matrix_file)
gene_annotation = get_gene_annotation(soft_file)
gene_annotation['Gene'] = gene_annotation['gene_assignment'].apply(lambda x: extract_gene_symbols(x))
mapping_df = gene_annotation[['ID', 'Gene']].copy().dropna()
gene_data = apply_gene_mapping(gene_data_raw, mapping_df)
print(f"Retrieved gene expression data with shape: {gene_data.shape}")

# 4. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols in the expression data...")
try:
    # Normalize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    print(f"Normalized gene data shape: {normalized_gene_data.shape}")
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 5. Extract clinical data since it doesn't exist yet
print("\nExtracting clinical data...")
try:
    # Since we haven't yet extracted clinical features, do it now
    # First, read background information and clinical data from the matrix file
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Define conversion functions for trait (Down Syndrome), age, and gender
    def convert_trait(value):
        """Convert trait value (Down Syndrome status) to binary (0 for control, 1 for DS)"""
        if value is None:
            return None
        # Extract the value after colon and strip whitespace
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if value.upper() == 'DS':
            return 1
        elif value.upper() == 'CTL':
            return 0
        return None

    def convert_age(value):
        """Convert age to a numeric value in years"""
        if value is None:
            return None
        
        # Extract the value after colon and strip whitespace
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Extract numeric part and unit
        import re
        match = re.match(r'(\d+)(\w+)', value)
        if not match:
            return None
        
        number, unit = match.groups()
        number = float(number)
        
        # Convert to years based on unit
        if unit == 'yr':
            return number
        elif unit == 'mo':
            return number / 12
        elif unit == 'wg':  # weeks of gestation
            return number / 52  # approximate conversion to years
        
        return None

    def convert_gender(value):
        """Convert gender to binary (0 for female, 1 for male)"""
        if value is None:
            return None
        
        # Extract the value after colon and strip whitespace
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if value.upper() == 'F':
            return 0
        elif value.upper() == 'M':
            return 1
        
        return None
    
    # Based on earlier inspection, we know:
    # trait_row = 2  (disease status: CTL or DS)
    # age_row = 4    (age: with various values)
    # gender_row = 3 (Sex: F or M)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_data, 
        trait=trait, 
        trait_row=2, 
        convert_trait=convert_trait,
        age_row=4, 
        convert_age=convert_age, 
        gender_row=3, 
        convert_gender=convert_gender
    )
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data extracted and saved to {out_clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    selected_clinical_df = pd.DataFrame()  # Empty dataframe instead of None
    is_trait_available = False

# 6. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and is_gene_available:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 7. Handle missing values systematically
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, data shape: {linked_data.shape}")
        
        # 8. Determine whether features are biased
        print("\nChecking for bias in features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    else:
        print("Cannot link data: gene or trait data is not available")
        if is_trait_available:
            linked_data = selected_clinical_df.T  # Use clinical data if available
        else:
            linked_data = pd.DataFrame({trait: [0, 1]})  # Minimal dataframe for validation
        is_biased = True
except Exception as e:
    print(f"Error in data linking or processing: {e}")
    linked_data = pd.DataFrame({trait: [0, 1]})  # Minimal dataframe for validation
    is_biased = True

# 9. Validate and save cohort info
print("\nPerforming final validation...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Down Syndrome dataset with gene expression from brain tissues across lifespan."
)

# 10. Save the linked data if usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")